# pyGSK — Monte Carlo SK & Pearson Fits Overlay

In [ ]:
import numpy as np, matplotlib.pyplot as plt
from scipy import stats
from pygsk.core import get_sk
from pygsk.thresholds import compute_sk_thresholds, PTypeIV, _normalizer_typeIV
def pdf_typeIII(x, mu, sigma, gamma1):
    return stats.pearson3.pdf(x, skew=gamma1, loc=mu, scale=sigma)
def pdf_typeI(x, p, q, a, b):
    return stats.beta.pdf(x, a=p, b=q, loc=a, scale=(b-a))
def pdf_typeVI(x, p, q, loc, scale):
    return stats.betaprime.pdf(x, a=p, b=q, loc=loc, scale=scale)
def pdf_typeIV(x, m, nu, loc, scale):
    params = PTypeIV(m=float(m), nu=float(nu), loc=float(loc), scale=float(scale))
    Ctheta = _normalizer_typeIV(params)
    t = (x - params.loc)/params.scale
    return (Ctheta/params.scale) * (1.0 + t*t)**(-params.m) * np.exp(params.nu*np.arctan(t))
M,N,d = 128,64,1.0
pfa = 1e-3
ns, seed = 50000, 123
rng = np.random.default_rng(seed)
samples = rng.gamma(shape=N, scale=1.0, size=(ns, M))
s1 = samples.sum(axis=1); s2 = (samples**2).sum(axis=1)
sk = get_sk(s1, s2, M, N, d)
x_min = max(0, np.percentile(sk, 0.02)); x_max = np.percentile(sk, 99.98)
pad = 0.1*(x_max-x_min); x = np.linspace(x_min-pad, x_max+pad, 1000)
style = {'I':{'color':'#1f77b4','label':'Type I'}, 'III':{'color':'#2ca02c','label':'Type III'}, 'IV':{'color':'#9467bd','label':'Type IV'}, 'VI':{'color':'#ff7f0e','label':'Type VI'}}
pdfs, thresholds = {}, {}
lok, hik, _, metak = compute_sk_thresholds(M, N, d, pfa, return_meta=True, mode='kappa', kappa_eps=1e-6)
fam_used = metak['family']
lo3, hi3, std3, meta3 = compute_sk_thresholds(M, N, d, pfa, return_meta=True, mode='explicit', family='III')
thresholds['III']=(lo3,hi3); pdfs['III']=pdf_typeIII(x,1.0,std3,meta3['gamma1'])
loi, hii, _, metai = compute_sk_thresholds(M, N, d, pfa, return_meta=True, mode='explicit', family='I')
thresholds['I']=(loi,hii); pdfs['I']=pdf_typeI(x,metai['params']['p'],metai['params']['q'],metai['params']['a'],metai['params']['b'])
lovi, hivi, _, metav = compute_sk_thresholds(M, N, d, pfa, return_meta=True, mode='explicit', family='VI')
thresholds['VI']=(lovi,hivi); pdfs['VI']=pdf_typeVI(x,metav['params']['p'],metav['params']['q'],metav['params']['loc'],metav['params']['scale'])
loiv, hiiv, _, meta4 = compute_sk_thresholds(M, N, d, pfa, return_meta=True, mode='explicit', family='IV')
thresholds['IV']=(loiv,hiiv); pdfs['IV']=pdf_typeIV(x,meta4['params']['m'],meta4['params']['nu'],meta4['params']['loc'],meta4['params']['scale'])
fig, ax = plt.subplots(figsize=(9,5.5))
ax.hist(sk, bins=200, density=True, alpha=0.35, label='SK histogram')
pdf_handles=[]
for fam in ['I','III','IV','VI']:
    lw = 2.8 if fam==fam_used else 1.6
    lo,hi = thresholds[fam]
    lab = f"{style[fam]['label']}  (thr: {lo:.3f}, {hi:.3f})"
    h,=ax.plot(x,pdfs[fam],color=style[fam]['color'],lw=lw,label=lab); pdf_handles.append(h)
    ls='-.' if fam==fam_used else '--'
    ax.axvline(lo,color=style[fam]['color'],linestyle=ls,lw=1.2,alpha=0.9)
    ax.axvline(hi,color=style[fam]['color'],linestyle=ls,lw=1.2,alpha=0.9)
ax.set_title(f"SK fits vs Monte Carlo (M={M}, N={N}, d={d}, ns={ns})\nκ-selected: Type {fam_used} (one-sided pfa={pfa:g}, total≈{2*pfa:g})")
ax.set_xlabel('SK'); ax.set_ylabel('Density'); ax.set_yscale('log')
handles=[plt.Rectangle((0,0),1,1,fc='C0',alpha=0.35,label='SK histogram')]+pdf_handles
labels=[h.get_label() for h in handles]
ax.legend(handles,labels,loc='upper right',ncol=1,frameon=True,bbox_to_anchor=(1.02,1.0),borderaxespad=0.5)
lines=["Empirical PFA vs thresholds"]
for fam in ['I','III','IV','VI']:
    lo,hi=thresholds[fam]
    below=int(np.sum(sk<lo)); above=int(np.sum(sk>hi)); total=sk.size
    pfa_emp=(below+above)/total
    lines.append(f"Type {fam}: PFA={pfa_emp:.4g}  (b={below}, a={above})")
ax.text(0.02,0.98,"\n".join(lines),transform=ax.transAxes,va='top',ha='left',fontsize=9,family='monospace',bbox=dict(boxstyle='round',facecolor='white',alpha=0.85,lw=0.5))
fig.tight_layout(); plt.show()